In [1]:
import pandas as pd
import numpy as np
import json 
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient 
import time

In [2]:
owner = "dolthub"
database = "us-housing-prices-v2"  # Put the database
branch = "main"


# Set the query to retrieve the data
query = '''SELECT COUNT(*) FROM sales'''
res = requests.get(
  'https://www.dolthub.com/api/v1alpha1/{}/{}/{}'.format(owner, database, branch),
  params={'q': query},
  )

response = res.json()
total_count = int(response['rows'][0]['COUNT(*)'])
print(total_count)

106642382


In [5]:
import requests
import pandas as pd
from io import StringIO
from time import sleep
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# Azure Storage account connection string
connect_str = '

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Name of the container to store your data
container_name = 'nyhomes'

# Create the container if it doesn't exist
container_client = blob_service_client.get_container_client(container_name)
try:
    container_client.create_container()
except Exception as e:
    print(f"Container already exists: {e}")

# Define the increment value
increment = 1000

# Starting offset
offset = 1722000

# Placeholder for the base SQL query, with an OFFSET placeholder
base_query = "SELECT * FROM sales WHERE state = 'NY' LIMIT 1000 OFFSET {offset}"

# Define the maximum number of retries
max_retries = 3

while True:
    # Update the SQL query with the current offset
    updated_query = base_query.format(offset=offset)
    print(updated_query)  # Print the updated query

    # Initialize values to handle cases where the request fails
    values = None

    # Attempt to make the API request with retries
    for attempt in range(max_retries):
        try:
            res = requests.get('https://www.dolthub.com/api/v1alpha1/{}/{}/{}'.format(owner, database, branch), params={'q': updated_query})
            res.raise_for_status()  # Raise an exception for HTTP errors
            values = res.json()
            break  # If successful, break out of the retry loop
        except requests.exceptions.RequestException as e:
            print(f"Request failed (attempt {attempt + 1}): {e}")
            if attempt < max_retries - 1:
                sleep(2 ** attempt)  # Exponential backoff: 1s, 2s, 4s...
            else:
                print("Max retries reached, exiting.")
                exit(1)

    # Process the results if values is not None
    if values and "rows" in values:
        result = values['rows']
        if not result:  # Check if the result is empty
            print("No more data to process.")
            break

        # Convert the result to a DataFrame
        ny_df = pd.DataFrame(result)

        # Convert DataFrame to CSV in memory
        csv_buffer = StringIO()
        ny_df.to_csv(csv_buffer, index=False)

        # Create a unique blob name
        blob_name = f"sales_data_batch_{offset}.csv"

        # Get the blob client
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        # Upload the CSV data to Azure Blob Storage
        blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)
        print(f"Batch {offset} uploaded to Azure Blob Storage.")
    else:
        print("No 'rows' key in the response or request failed.")
        break

    # Increment the offset for the next iteration
    offset += increment

# Print the completion message
print("All data has been processed and uploaded to Azure Blob Storage.")

Container already exists: The specified container already exists.
RequestId:472c2d89-101e-006c-5168-a662bb000000
Time:2024-05-15T01:34:18.0751499Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:472c2d89-101e-006c-5168-a662bb000000
Time:2024-05-15T01:34:18.0751499Z</Message></Error>
SELECT * FROM sales WHERE state = 'NY' LIMIT 1000 OFFSET 1722000
Request failed (attempt 1): 504 Server Error: Gateway Timeout for url: https://www.dolthub.com/api/v1alpha1/dolthub/us-housing-prices-v2/main?q=SELECT+%2A+FROM+sales+WHERE+state+%3D+%27NY%27+LIMIT+1000+OFFSET+1722000
Request failed (attempt 2): 504 Server Error: Gateway Timeout for url: https://www.dolthub.com/api/v1alpha1/dolthub/us-housing-prices-v2/main?q=SELECT+%2A+FROM+sales+WHERE+state+%3D+%27NY%27+LIMIT+1000+OFFSET+1722000
Request failed (attempt 3): 504 Server Error: Gateway Timeout for url: https://www.d

In [6]:
import requests
import pandas as pd
from io import StringIO
from time import sleep
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# Azure Storage account connection string
connect_str = '
# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Name of the container to store your data
container_name = 'housingsales'

# Create the container if it doesn't exist
container_client = blob_service_client.get_container_client(container_name)
try:
    container_client.create_container()
except Exception as e:
    print(f"Container already exists: {e}")

# Define the increment value
increment = 1000

# Starting offset
offset = 0

# Placeholder for the base SQL query, with an OFFSET placeholder
base_query = "SELECT * FROM sales WHERE state = 'CT' LIMIT 1000 OFFSET {offset}"

# Define the maximum number of retries
max_retries = 3

while True:
    # Update the SQL query with the current offset
    updated_query = base_query.format(offset=offset)
    print(updated_query)  # Print the updated query

    # Initialize values to handle cases where the request fails
    values = None

    # Attempt to make the API request with retries
    for attempt in range(max_retries):
        try:
            res = requests.get('https://www.dolthub.com/api/v1alpha1/{}/{}/{}'.format(owner, database, branch), params={'q': updated_query})
            res.raise_for_status()  # Raise an exception for HTTP errors
            values = res.json()
            break  # If successful, break out of the retry loop
        except requests.exceptions.RequestException as e:
            print(f"Request failed (attempt {attempt + 1}): {e}")
            if attempt < max_retries - 1:
                sleep(2 ** attempt)  # Exponential backoff: 1s, 2s, 4s...
            else:
                print("Max retries reached, exiting.")
                exit(1)

    # Process the results if values is not None
    if values and "rows" in values:
        result = values['rows']
        if not result:  # Check if the result is empty
            print("No more data to process.")
            break

        # Convert the result to a DataFrame
        ct_df = pd.DataFrame(result)

        # Convert DataFrame to CSV in memory
        csv_buffer = StringIO()
        ct_df.to_csv(csv_buffer, index=False)

        # Create a unique blob name
        blob_name = f"sales_data_batch_{offset}.csv"

        # Get the blob client
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        # Upload the CSV data to Azure Blob Storage
        blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)
        print(f"Batch {offset} uploaded to Azure Blob Storage.")
    else:
        print("No 'rows' key in the response or request failed.")
        break

    # Increment the offset for the next iteration
    offset += increment

# Print the completion message
print("All data has been processed and uploaded to Azure Blob Storage.")

Container already exists: The specified container already exists.
RequestId:cb7bce5b-301e-0036-1569-a6045c000000
Time:2024-05-15T01:43:55.0923712Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:cb7bce5b-301e-0036-1569-a6045c000000
Time:2024-05-15T01:43:55.0923712Z</Message></Error>
SELECT * FROM sales WHERE state = 'CT' LIMIT 1000 OFFSET 0
Batch 0 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'CT' LIMIT 1000 OFFSET 1000
Batch 1000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'CT' LIMIT 1000 OFFSET 2000
Batch 2000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'CT' LIMIT 1000 OFFSET 3000
Batch 3000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'CT' LIMIT 1000 OFFSET 4000
Batch 4000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'CT' LIMIT 1000 OFFSET 5000
Ba

In [7]:
import requests
import pandas as pd
from io import StringIO
from time import sleep
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# Azure Storage account connection string
connect_str = '0

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Name of the container to store your data
container_name = 'housingsales'

# Create the container if it doesn't exist
container_client = blob_service_client.get_container_client(container_name)
try:
    container_client.create_container()
except Exception as e:
    print(f"Container already exists: {e}")

# Define the increment value
increment = 1000

# Starting offset
offset = 0

# Placeholder for the base SQL query, with an OFFSET placeholder
base_query = "SELECT * FROM sales WHERE state = 'VA' LIMIT 1000 OFFSET {offset}"

# Define the maximum number of retries
max_retries = 3

while True:
    # Update the SQL query with the current offset
    updated_query = base_query.format(offset=offset)
    print(updated_query)  # Print the updated query

    # Initialize values to handle cases where the request fails
    values = None

    # Attempt to make the API request with retries
    for attempt in range(max_retries):
        try:
            res = requests.get('https://www.dolthub.com/api/v1alpha1/{}/{}/{}'.format(owner, database, branch), params={'q': updated_query})
            res.raise_for_status()  # Raise an exception for HTTP errors
            values = res.json()
            break  # If successful, break out of the retry loop
        except requests.exceptions.RequestException as e:
            print(f"Request failed (attempt {attempt + 1}): {e}")
            if attempt < max_retries - 1:
                sleep(2 ** attempt)  # Exponential backoff: 1s, 2s, 4s...
            else:
                print("Max retries reached, exiting.")
                exit(1)

    # Process the results if values is not None
    if values and "rows" in values:
        result = values['rows']
        if not result:  # Check if the result is empty
            print("No more data to process.")
            break

        # Convert the result to a DataFrame
        va_df = pd.DataFrame(result)

        # Convert DataFrame to CSV in memory
        csv_buffer = StringIO()
        va_df.to_csv(csv_buffer, index=False)

        # Create a unique blob name
        blob_name = f"sales_data_batch_{offset}.csv"

        # Get the blob client
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        # Upload the CSV data to Azure Blob Storage
        blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)
        print(f"Batch {offset} uploaded to Azure Blob Storage.")
    else:
        print("No 'rows' key in the response or request failed.")
        break

    # Increment the offset for the next iteration
    offset += increment

# Print the completion message
print("All data has been processed and uploaded to Azure Blob Storage.")

Container already exists: The specified container already exists.
RequestId:476d4ed4-a01e-001b-207c-a6b72f000000
Time:2024-05-15T04:02:02.1689814Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:476d4ed4-a01e-001b-207c-a6b72f000000
Time:2024-05-15T04:02:02.1689814Z</Message></Error>
SELECT * FROM sales WHERE state = 'VA' LIMIT 1000 OFFSET 0
Batch 0 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'VA' LIMIT 1000 OFFSET 1000
Batch 1000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'VA' LIMIT 1000 OFFSET 2000
Batch 2000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'VA' LIMIT 1000 OFFSET 3000
Batch 3000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'VA' LIMIT 1000 OFFSET 4000
Batch 4000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'VA' LIMIT 1000 OFFSET 5000
Ba

In [3]:
import requests
import pandas as pd
from io import StringIO
from time import sleep
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# Azure Storage account connection string
connect_str = '0

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Name of the container to store your data
container_name = 'housingsales'

# Create the container if it doesn't exist
container_client = blob_service_client.get_container_client(container_name)
try:
    container_client.create_container()
except Exception as e:
    print(f"Container already exists: {e}")

# Define the increment value
increment = 1000

# Starting offset
offset = 14000

# Placeholder for the base SQL query, with an OFFSET placeholder
base_query = "SELECT * FROM sales WHERE state = 'MD' LIMIT 1000 OFFSET {offset}"

# Define the maximum number of retries
max_retries = 3

while True:
    # Update the SQL query with the current offset
    updated_query = base_query.format(offset=offset)
    print(updated_query)  # Print the updated query

    # Initialize values to handle cases where the request fails
    values = None

    # Attempt to make the API request with retries
    for attempt in range(max_retries):
        try:
            res = requests.get('https://www.dolthub.com/api/v1alpha1/{}/{}/{}'.format(owner, database, branch), params={'q': updated_query})
            res.raise_for_status()  # Raise an exception for HTTP errors
            values = res.json()
            break  # If successful, break out of the retry loop
        except requests.exceptions.RequestException as e:
            print(f"Request failed (attempt {attempt + 1}): {e}")
            if attempt < max_retries - 1:
                sleep(2 ** attempt)  # Exponential backoff: 1s, 2s, 4s...
            else:
                print("Max retries reached, exiting.")
                exit(1)

    # Process the results if values is not None
    if values and "rows" in values:
        result = values['rows']
        if not result:  # Check if the result is empty
            print("No more data to process.")
            break

        # Convert the result to a DataFrame
        md_df = pd.DataFrame(result)

        # Convert DataFrame to CSV in memory
        csv_buffer = StringIO()
        md_df.to_csv(csv_buffer, index=False)

        # Create a unique blob name
        blob_name = f"sales_data_batch_{offset}.csv"

        # Get the blob client
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        # Upload the CSV data to Azure Blob Storage
        blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)
        print(f"Batch {offset} uploaded to Azure Blob Storage.")
    else:
        print("No 'rows' key in the response or request failed.")
        break

    # Increment the offset for the next iteration
    offset += increment

# Print the completion message
print("All data has been processed and uploaded to Azure Blob Storage.")

Container already exists: The specified container already exists.
RequestId:96a38a6d-f01e-004b-4100-a7757f000000
Time:2024-05-15T19:44:49.9742054Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:96a38a6d-f01e-004b-4100-a7757f000000
Time:2024-05-15T19:44:49.9742054Z</Message></Error>
SELECT * FROM sales WHERE state = 'MD' LIMIT 1000 OFFSET 14000
Request failed (attempt 1): 504 Server Error: Gateway Timeout for url: https://www.dolthub.com/api/v1alpha1/dolthub/us-housing-prices-v2/main?q=SELECT+%2A+FROM+sales+WHERE+state+%3D+%27MD%27+LIMIT+1000+OFFSET+14000
Request failed (attempt 2): 504 Server Error: Gateway Timeout for url: https://www.dolthub.com/api/v1alpha1/dolthub/us-housing-prices-v2/main?q=SELECT+%2A+FROM+sales+WHERE+state+%3D+%27MD%27+LIMIT+1000+OFFSET+14000
Request failed (attempt 3): 504 Server Error: Gateway Timeout for url: https://www.dolthub

In [3]:
import requests
import pandas as pd
from io import StringIO
from time import sleep
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# Azure Storage account connection string
connect_str = '0

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Name of the container to store your data
container_name = 'housingsales'

# Create the container if it doesn't exist
container_client = blob_service_client.get_container_client(container_name)
try:
    container_client.create_container()
except Exception as e:
    print(f"Container already exists: {e}")

# Define the increment value
increment = 1000

# Starting offset
offset = 140000

# Placeholder for the base SQL query, with an OFFSET placeholder
base_query = "SELECT * FROM sales WHERE state = 'CA' LIMIT 1000 OFFSET {offset}"

# Define the maximum number of retries
max_retries = 3

while True:
    # Update the SQL query with the current offset
    updated_query = base_query.format(offset=offset)
    print(updated_query)  # Print the updated query

    # Initialize values to handle cases where the request fails
    values = None

    # Attempt to make the API request with retries
    for attempt in range(max_retries):
        try:
            res = requests.get('https://www.dolthub.com/api/v1alpha1/{}/{}/{}'.format(owner, database, branch), params={'q': updated_query})
            res.raise_for_status()  # Raise an exception for HTTP errors
            values = res.json()
            break  # If successful, break out of the retry loop
        except requests.exceptions.RequestException as e:
            print(f"Request failed (attempt {attempt + 1}): {e}")
            if attempt < max_retries - 1:
                sleep(2 ** attempt)  # Exponential backoff: 1s, 2s, 4s...
            else:
                print("Max retries reached, exiting.")
                exit(1)

    # Process the results if values is not None
    if values and "rows" in values:
        result = values['rows']
        if not result:  # Check if the result is empty
            print("No more data to process.")
            break

        # Convert the result to a DataFrame
        ca_df = pd.DataFrame(result)

        # Convert DataFrame to CSV in memory
        csv_buffer = StringIO()
        ca_df.to_csv(csv_buffer, index=False)

        # Create a unique blob name
        blob_name = f"sales_data_batch_{offset}.csv"

        # Get the blob client
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        # Upload the CSV data to Azure Blob Storage
        blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)
        print(f"Batch {offset} uploaded to Azure Blob Storage.")
    else:
        print("No 'rows' key in the response or request failed.")
        break

    # Increment the offset for the next iteration
    offset += increment

# Print the completion message
print("All data has been processed and uploaded to Azure Blob Storage.")

Container already exists: The specified container already exists.
RequestId:96ebdbbe-f01e-004b-7411-a7757f000000
Time:2024-05-15T21:45:09.1309578Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:96ebdbbe-f01e-004b-7411-a7757f000000
Time:2024-05-15T21:45:09.1309578Z</Message></Error>
SELECT * FROM sales WHERE state = 'CA' LIMIT 1000 OFFSET 140000
Request failed (attempt 1): 504 Server Error: Gateway Timeout for url: https://www.dolthub.com/api/v1alpha1/dolthub/us-housing-prices-v2/main?q=SELECT+%2A+FROM+sales+WHERE+state+%3D+%27CA%27+LIMIT+1000+OFFSET+140000
Request failed (attempt 2): 504 Server Error: Gateway Timeout for url: https://www.dolthub.com/api/v1alpha1/dolthub/us-housing-prices-v2/main?q=SELECT+%2A+FROM+sales+WHERE+state+%3D+%27CA%27+LIMIT+1000+OFFSET+140000
Request failed (attempt 3): 504 Server Error: Gateway Timeout for url: https://www.dolt

In [3]:
import requests
import pandas as pd
from io import StringIO
from time import sleep
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# Azure Storage account connection string
connect_str = '0

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Name of the container to store your data
container_name = 'housingsales'

# Create the container if it doesn't exist
container_client = blob_service_client.get_container_client(container_name)
try:
    container_client.create_container()
except Exception as e:
    print(f"Container already exists: {e}")

# Define the increment value
increment = 1000

# Starting offset
offset = 574000

# Placeholder for the base SQL query, with an OFFSET placeholder
base_query = "SELECT * FROM sales WHERE state = 'TX' LIMIT 1000 OFFSET {offset}"

# Define the maximum number of retries
max_retries = 3

while True:
    # Update the SQL query with the current offset
    updated_query = base_query.format(offset=offset)
    print(updated_query)  # Print the updated query

    # Initialize values to handle cases where the request fails
    values = None

    # Attempt to make the API request with retries
    for attempt in range(max_retries):
        try:
            res = requests.get('https://www.dolthub.com/api/v1alpha1/{}/{}/{}'.format(owner, database, branch), params={'q': updated_query})
            res.raise_for_status()  # Raise an exception for HTTP errors
            values = res.json()
            break  # If successful, break out of the retry loop
        except requests.exceptions.RequestException as e:
            print(f"Request failed (attempt {attempt + 1}): {e}")
            if attempt < max_retries - 1:
                sleep(2 ** attempt)  # Exponential backoff: 1s, 2s, 4s...
            else:
                print("Max retries reached, exiting.")
                exit(1)

    # Process the results if values is not None
    if values and "rows" in values:
        result = values['rows']
        if not result:  # Check if the result is empty
            print("No more data to process.")
            break

        # Convert the result to a DataFrame
        tx_df = pd.DataFrame(result)

        # Convert DataFrame to CSV in memory
        csv_buffer = StringIO()
        tx_df.to_csv(csv_buffer, index=False)

        # Create a unique blob name
        blob_name = f"sales_data_batch_{offset}.csv"

        # Get the blob client
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        # Upload the CSV data to Azure Blob Storage
        blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)
        print(f"Batch {offset} uploaded to Azure Blob Storage.")
    else:
        print("No 'rows' key in the response or request failed.")
        break

    # Increment the offset for the next iteration
    offset += increment

# Print the completion message
print("All data has been processed and uploaded to Azure Blob Storage.")

Container already exists: The specified container already exists.
RequestId:02379371-301e-0044-551d-a70313000000
Time:2024-05-15T23:16:12.9517261Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:02379371-301e-0044-551d-a70313000000
Time:2024-05-15T23:16:12.9517261Z</Message></Error>
SELECT * FROM sales WHERE state = 'TX' LIMIT 1000 OFFSET 574000
Request failed (attempt 1): 504 Server Error: Gateway Timeout for url: https://www.dolthub.com/api/v1alpha1/dolthub/us-housing-prices-v2/main?q=SELECT+%2A+FROM+sales+WHERE+state+%3D+%27TX%27+LIMIT+1000+OFFSET+574000
Batch 574000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'TX' LIMIT 1000 OFFSET 575000
Batch 575000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'TX' LIMIT 1000 OFFSET 576000
Batch 576000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'TX' L

In [3]:
import requests
import pandas as pd
from io import StringIO
from time import sleep
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# Azure Storage account connection string
connect_str = '0

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Name of the container to store your data
container_name = 'housingsales'

# Create the container if it doesn't exist
container_client = blob_service_client.get_container_client(container_name)
try:
    container_client.create_container()
except Exception as e:
    print(f"Container already exists: {e}")

# Define the increment value
increment = 1000

# Starting offset
offset = 0

# Placeholder for the base SQL query, with an OFFSET placeholder
base_query = "SELECT * FROM sales WHERE state = 'AZ' LIMIT 1000 OFFSET {offset}"

# Define the maximum number of retries
max_retries = 3

while True:
    # Update the SQL query with the current offset
    updated_query = base_query.format(offset=offset)
    print(updated_query)  # Print the updated query

    # Initialize values to handle cases where the request fails
    values = None

    # Attempt to make the API request with retries
    for attempt in range(max_retries):
        try:
            res = requests.get('https://www.dolthub.com/api/v1alpha1/{}/{}/{}'.format(owner, database, branch), params={'q': updated_query})
            res.raise_for_status()  # Raise an exception for HTTP errors
            values = res.json()
            break  # If successful, break out of the retry loop
        except requests.exceptions.RequestException as e:
            print(f"Request failed (attempt {attempt + 1}): {e}")
            if attempt < max_retries - 1:
                sleep(2 ** attempt)  # Exponential backoff: 1s, 2s, 4s...
            else:
                print("Max retries reached, exiting.")
                exit(1)

    # Process the results if values is not None
    if values and "rows" in values:
        result = values['rows']
        if not result:  # Check if the result is empty
            print("No more data to process.")
            break

        # Convert the result to a DataFrame
        az_df = pd.DataFrame(result)

        # Convert DataFrame to CSV in memory
        csv_buffer = StringIO()
        az_df.to_csv(csv_buffer, index=False)

        # Create a unique blob name
        blob_name = f"sales_data_batch_{offset}.csv"

        # Get the blob client
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        # Upload the CSV data to Azure Blob Storage
        blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)
        print(f"Batch {offset} uploaded to Azure Blob Storage.")
    else:
        print("No 'rows' key in the response or request failed.")
        break

    # Increment the offset for the next iteration
    offset += increment

# Print the completion message
print("All data has been processed and uploaded to Azure Blob Storage.")

Container already exists: The specified container already exists.
RequestId:1e84fde5-901e-0010-2c2d-a74c44000000
Time:2024-05-16T01:09:41.4978939Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:1e84fde5-901e-0010-2c2d-a74c44000000
Time:2024-05-16T01:09:41.4978939Z</Message></Error>
SELECT * FROM sales WHERE state = 'AZ' LIMIT 1000 OFFSET 0
Batch 0 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'AZ' LIMIT 1000 OFFSET 1000
Batch 1000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'AZ' LIMIT 1000 OFFSET 2000
Batch 2000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'AZ' LIMIT 1000 OFFSET 3000
Batch 3000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'AZ' LIMIT 1000 OFFSET 4000
Batch 4000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'AZ' LIMIT 1000 OFFSET 5000
Ba

In [6]:
import requests
import pandas as pd
from io import StringIO
from time import sleep
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# Azure Storage account connection string
connect_str = '0

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Name of the container to store your data
container_name = 'housingsales'

# Create the container if it doesn't exist
container_client = blob_service_client.get_container_client(container_name)
try:
    container_client.create_container()
except Exception as e:
    print(f"Container already exists: {e}")

# Define the increment value
increment = 1000

# Starting offset
offset = 0

# Placeholder for the base SQL query, with an OFFSET placeholder
base_query = "SELECT * FROM sales WHERE state = 'FL' LIMIT 1000 OFFSET {offset}"

# Define the maximum number of retries
max_retries = 3

while True:
    # Update the SQL query with the current offset
    updated_query = base_query.format(offset=offset)
    print(updated_query)  # Print the updated query

    # Initialize values to handle cases where the request fails
    values = None

    # Attempt to make the API request with retries
    for attempt in range(max_retries):
        try:
            res = requests.get('https://www.dolthub.com/api/v1alpha1/{}/{}/{}'.format(owner, database, branch), params={'q': updated_query})
            res.raise_for_status()  # Raise an exception for HTTP errors
            values = res.json()
            break  # If successful, break out of the retry loop
        except requests.exceptions.RequestException as e:
            print(f"Request failed (attempt {attempt + 1}): {e}")
            if attempt < max_retries - 1:
                sleep(2 ** attempt)  # Exponential backoff: 1s, 2s, 4s...
            else:
                print("Max retries reached, exiting.")
                exit(1)

    # Process the results if values is not None
    if values and "rows" in values:
        result = values['rows']
        if not result:  # Check if the result is empty
            print("No more data to process.")
            break

        # Convert the result to a DataFrame
        fl_df = pd.DataFrame(result)

        # Convert DataFrame to CSV in memory
        csv_buffer = StringIO()
        fl_df.to_csv(csv_buffer, index=False)

        # Create a unique blob name
        blob_name = f"sales_data_batch_{offset}.csv"

        # Get the blob client
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        # Upload the CSV data to Azure Blob Storage
        blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)
        print(f"Batch {offset} uploaded to Azure Blob Storage.")
    else:
        print("No 'rows' key in the response or request failed.")
        break

    # Increment the offset for the next iteration
    offset += increment

# Print the completion message
print("All data has been processed and uploaded to Azure Blob Storage.")

Container already exists: The specified container already exists.
RequestId:c9fac03d-601e-003b-1332-a7cc88000000
Time:2024-05-16T01:45:38.2715652Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:c9fac03d-601e-003b-1332-a7cc88000000
Time:2024-05-16T01:45:38.2715652Z</Message></Error>
SELECT * FROM sales WHERE state = 'FL' LIMIT 1000 OFFSET 0
Batch 0 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'FL' LIMIT 1000 OFFSET 1000
Batch 1000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'FL' LIMIT 1000 OFFSET 2000
Batch 2000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'FL' LIMIT 1000 OFFSET 3000
Batch 3000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'FL' LIMIT 1000 OFFSET 4000
Batch 4000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'FL' LIMIT 1000 OFFSET 5000
Ba

In [3]:
import requests
import pandas as pd
from io import StringIO
from time import sleep
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# Azure Storage account connection string
connect_str = '0

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Name of the container to store your data
container_name = 'housingsales'

# Create the container if it doesn't exist
container_client = blob_service_client.get_container_client(container_name)
try:
    container_client.create_container()
except Exception as e:
    print(f"Container already exists: {e}")

# Define the increment value
increment = 1000

# Starting offset
offset = 481000

# Placeholder for the base SQL query, with an OFFSET placeholder
base_query = "SELECT * FROM sales WHERE state = 'NJ' LIMIT 1000 OFFSET {offset}"

# Define the maximum number of retries
max_retries = 3

while True:
    # Update the SQL query with the current offset
    updated_query = base_query.format(offset=offset)
    print(updated_query)  # Print the updated query

    # Initialize values to handle cases where the request fails
    values = None

    # Attempt to make the API request with retries
    for attempt in range(max_retries):
        try:
            res = requests.get('https://www.dolthub.com/api/v1alpha1/{}/{}/{}'.format(owner, database, branch), params={'q': updated_query})
            res.raise_for_status()  # Raise an exception for HTTP errors
            values = res.json()
            break  # If successful, break out of the retry loop
        except requests.exceptions.RequestException as e:
            print(f"Request failed (attempt {attempt + 1}): {e}")
            if attempt < max_retries - 1:
                sleep(2 ** attempt)  # Exponential backoff: 1s, 2s, 4s...
            else:
                print("Max retries reached, exiting.")
                exit(1)

    # Process the results if values is not None
    if values and "rows" in values:
        result = values['rows']
        if not result:  # Check if the result is empty
            print("No more data to process.")
            break

        # Convert the result to a DataFrame
        nj_df = pd.DataFrame(result)

        # Convert DataFrame to CSV in memory
        csv_buffer = StringIO()
        nj_df.to_csv(csv_buffer, index=False)

        # Create a unique blob name
        blob_name = f"sales_data_batch_{offset}.csv"

        # Get the blob client
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        # Upload the CSV data to Azure Blob Storage
        blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)
        print(f"Batch {offset} uploaded to Azure Blob Storage.")
    else:
        print("No 'rows' key in the response or request failed.")
        break

    # Increment the offset for the next iteration
    offset += increment

# Print the completion message
print("All data has been processed and uploaded to Azure Blob Storage.")

Container already exists: The specified container already exists.
RequestId:75a05d04-901e-004d-34a8-a746c0000000
Time:2024-05-16T15:50:19.6814404Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:75a05d04-901e-004d-34a8-a746c0000000
Time:2024-05-16T15:50:19.6814404Z</Message></Error>
SELECT * FROM sales WHERE state = 'NJ' LIMIT 1000 OFFSET 481000
Request failed (attempt 1): 504 Server Error: Gateway Timeout for url: https://www.dolthub.com/api/v1alpha1/dolthub/us-housing-prices-v2/main?q=SELECT+%2A+FROM+sales+WHERE+state+%3D+%27NJ%27+LIMIT+1000+OFFSET+481000
Request failed (attempt 2): 504 Server Error: Gateway Timeout for url: https://www.dolthub.com/api/v1alpha1/dolthub/us-housing-prices-v2/main?q=SELECT+%2A+FROM+sales+WHERE+state+%3D+%27NJ%27+LIMIT+1000+OFFSET+481000
Request failed (attempt 3): 504 Server Error: Gateway Timeout for url: https://www.dolt

In [3]:
import requests
import pandas as pd
from io import StringIO
from time import sleep
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# Azure Storage account connection string
connect_str = '0

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Name of the container to store your data
container_name = 'housingsales'

# Create the container if it doesn't exist
container_client = blob_service_client.get_container_client(container_name)
try:
    container_client.create_container()
except Exception as e:
    print(f"Container already exists: {e}")

# Define the increment value
increment = 1000

# Starting offset
offset = 554000

# Placeholder for the base SQL query, with an OFFSET placeholder
base_query = "SELECT * FROM sales WHERE state = 'WA' LIMIT 1000 OFFSET {offset}"

# Define the maximum number of retries
max_retries = 3

while True:
    # Update the SQL query with the current offset
    updated_query = base_query.format(offset=offset)
    print(updated_query)  # Print the updated query

    # Initialize values to handle cases where the request fails
    values = None

    # Attempt to make the API request with retries
    for attempt in range(max_retries):
        try:
            res = requests.get('https://www.dolthub.com/api/v1alpha1/{}/{}/{}'.format(owner, database, branch), params={'q': updated_query})
            res.raise_for_status()  # Raise an exception for HTTP errors
            values = res.json()
            break  # If successful, break out of the retry loop
        except requests.exceptions.RequestException as e:
            print(f"Request failed (attempt {attempt + 1}): {e}")
            if attempt < max_retries - 1:
                sleep(2 ** attempt)  # Exponential backoff: 1s, 2s, 4s...
            else:
                print("Max retries reached, exiting.")
                exit(1)

    # Process the results if values is not None
    if values and "rows" in values:
        result = values['rows']
        if not result:  # Check if the result is empty
            print("No more data to process.")
            break

        # Convert the result to a DataFrame
        wa_df = pd.DataFrame(result)

        # Convert DataFrame to CSV in memory
        csv_buffer = StringIO()
        wa_df.to_csv(csv_buffer, index=False)

        # Create a unique blob name
        blob_name = f"sales_data_batch_{offset}.csv"

        # Get the blob client
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        # Upload the CSV data to Azure Blob Storage
        blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)
        print(f"Batch {offset} uploaded to Azure Blob Storage.")
    else:
        print("No 'rows' key in the response or request failed.")
        break

    # Increment the offset for the next iteration
    offset += increment

# Print the completion message
print("All data has been processed and uploaded to Azure Blob Storage.")

Container already exists: The specified container already exists.
RequestId:2db42b3b-901e-0000-5a0f-a8892c000000
Time:2024-05-17T04:05:07.4518855Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:2db42b3b-901e-0000-5a0f-a8892c000000
Time:2024-05-17T04:05:07.4518855Z</Message></Error>
SELECT * FROM sales WHERE state = 'WA' LIMIT 1000 OFFSET 554000
Request failed (attempt 1): 504 Server Error: Gateway Timeout for url: https://www.dolthub.com/api/v1alpha1/dolthub/us-housing-prices-v2/main?q=SELECT+%2A+FROM+sales+WHERE+state+%3D+%27WA%27+LIMIT+1000+OFFSET+554000
Request failed (attempt 2): 504 Server Error: Gateway Timeout for url: https://www.dolthub.com/api/v1alpha1/dolthub/us-housing-prices-v2/main?q=SELECT+%2A+FROM+sales+WHERE+state+%3D+%27WA%27+LIMIT+1000+OFFSET+554000
Request failed (attempt 3): 504 Server Error: Gateway Timeout for url: https://www.dolt

In [ ]:
import requests
import pandas as pd
from io import StringIO
from time import sleep
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# Azure Storage account connection string
connect_str = '0

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Name of the container to store your data
container_name = 'housingsales'

# Create the container if it doesn't exist
container_client = blob_service_client.get_container_client(container_name)
try:
    container_client.create_container()
except Exception as e:
    print(f"Container already exists: {e}")

# Define the increment value
increment = 1000

# Starting offset
offset = 0

# Placeholder for the base SQL query, with an OFFSET placeholder
base_query = "SELECT * FROM sales WHERE state = 'IL' LIMIT 1000 OFFSET {offset}"

# Define the maximum number of retries
max_retries = 3

while True:
    # Update the SQL query with the current offset
    updated_query = base_query.format(offset=offset)
    print(updated_query)  # Print the updated query

    # Initialize values to handle cases where the request fails
    values = None

    # Attempt to make the API request with retries
    for attempt in range(max_retries):
        try:
            res = requests.get('https://www.dolthub.com/api/v1alpha1/{}/{}/{}'.format(owner, database, branch), params={'q': updated_query})
            res.raise_for_status()  # Raise an exception for HTTP errors
            values = res.json()
            break  # If successful, break out of the retry loop
        except requests.exceptions.RequestException as e:
            print(f"Request failed (attempt {attempt + 1}): {e}")
            if attempt < max_retries - 1:
                sleep(2 ** attempt)  # Exponential backoff: 1s, 2s, 4s...
            else:
                print("Max retries reached, exiting.")
                exit(1)

    # Process the results if values is not None
    if values and "rows" in values:
        result = values['rows']
        if not result:  # Check if the result is empty
            print("No more data to process.")
            break

        # Convert the result to a DataFrame
        il_df = pd.DataFrame(result)

        # Convert DataFrame to CSV in memory
        csv_buffer = StringIO()
        il_df.to_csv(csv_buffer, index=False)

        # Create a unique blob name
        blob_name = f"sales_data_batch_{offset}.csv"

        # Get the blob client
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        # Upload the CSV data to Azure Blob Storage
        blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)
        print(f"Batch {offset} uploaded to Azure Blob Storage.")
    else:
        print("No 'rows' key in the response or request failed.")
        break

    # Increment the offset for the next iteration
    offset += increment

# Print the completion message
print("All data has been processed and uploaded to Azure Blob Storage.")

Container already exists: The specified container already exists.
RequestId:3ee21a78-001e-003d-420f-a8ff37000000
Time:2024-05-17T04:07:48.0857514Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:3ee21a78-001e-003d-420f-a8ff37000000
Time:2024-05-17T04:07:48.0857514Z</Message></Error>
SELECT * FROM sales WHERE state = 'IL' LIMIT 1000 OFFSET 0
Batch 0 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'IL' LIMIT 1000 OFFSET 1000
Batch 1000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'IL' LIMIT 1000 OFFSET 2000
Batch 2000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'IL' LIMIT 1000 OFFSET 3000
Batch 3000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'IL' LIMIT 1000 OFFSET 4000
Batch 4000 uploaded to Azure Blob Storage.
SELECT * FROM sales WHERE state = 'IL' LIMIT 1000 OFFSET 5000
Ba

In [ ]:
import requests
import pandas as pd
from io import StringIO
from time import sleep
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# Azure Storage account connection string
connect_str = '0

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Name of the container to store your data
container_name = 'housingsales'

# Create the container if it doesn't exist
container_client = blob_service_client.get_container_client(container_name)
try:
    container_client.create_container()
except Exception as e:
    print(f"Container already exists: {e}")

# Define the increment value
increment = 1000

# Starting offset
offset = 0

# Placeholder for the base SQL query, with an OFFSET placeholder
base_query = "SELECT * FROM sales WHERE state = 'NH' LIMIT 1000 OFFSET {offset}"

# Define the maximum number of retries
max_retries = 3

while True:
    # Update the SQL query with the current offset
    updated_query = base_query.format(offset=offset)
    print(updated_query)  # Print the updated query

    # Initialize values to handle cases where the request fails
    values = None

    # Attempt to make the API request with retries
    for attempt in range(max_retries):
        try:
            res = requests.get('https://www.dolthub.com/api/v1alpha1/{}/{}/{}'.format(owner, database, branch), params={'q': updated_query})
            res.raise_for_status()  # Raise an exception for HTTP errors
            values = res.json()
            break  # If successful, break out of the retry loop
        except requests.exceptions.RequestException as e:
            print(f"Request failed (attempt {attempt + 1}): {e}")
            if attempt < max_retries - 1:
                sleep(2 ** attempt)  # Exponential backoff: 1s, 2s, 4s...
            else:
                print("Max retries reached, exiting.")
                exit(1)

    # Process the results if values is not None
    if values and "rows" in values:
        result = values['rows']
        if not result:  # Check if the result is empty
            print("No more data to process.")
            break

        # Convert the result to a DataFrame
        nh_df = pd.DataFrame(result)

        # Convert DataFrame to CSV in memory
        csv_buffer = StringIO()
        nh_df.to_csv(csv_buffer, index=False)

        # Create a unique blob name
        blob_name = f"sales_data_batch_{offset}.csv"

        # Get the blob client
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        # Upload the CSV data to Azure Blob Storage
        blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)
        print(f"Batch {offset} uploaded to Azure Blob Storage.")
    else:
        print("No 'rows' key in the response or request failed.")
        break

    # Increment the offset for the next iteration
    offset += increment

# Print the completion message
print("All data has been processed and uploaded to Azure Blob Storage.")